In [ ]:
import numpy as np
import pandas as pd
import math
import sklearn.preprocessing
import datetime
from TimeBasedCV import TimeBasedCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import make_scorer, r2_score
import statsmodels.api as sm
import warnings
import pickle

warnings.simplefilter(action='ignore', category=Warning)
from sklearn.linear_model import HuberRegressor
# pd.set_option('display.max_rows', None)
# more

In [ ]:
# df = pd.read_csv('data/factors_1970.csv', parse_dates=['DATE'])
# df = pd.read_csv('data/features_subset.csv', parse_dates=['DATE'])
# df = pd.read_csv('factors_1900.csv', parse_dates=['DATE'])
# df.drop(columns=['sic2'], inplace=True)

In [ ]:
# with open('data/features_1975.pkl', 'wb') as f:
#     pickle.dump(df, f)

with open('data/features_1975.pkl', 'rb') as f:
    df = pickle.load(f)
    print(df.head())

In [ ]:
#Sort observations by date and stock id
df[df.columns[2:]] = df[df.columns[2:]].astype('float32')
df = df.sort_values(by = ['DATE', 'permno'], ascending = True)
df.head()


In [ ]:
df['permno2'] = df['permno'].copy()
df['DATE2'] = df['DATE'].copy()
df = df.set_index(['DATE2','permno2'])

#Make a copy of  the "me" variable (market equity) before rank standartization to use afterwards for value weighting
df['mvel12'] = df['mvel1'].copy()

In [ ]:
p=0.3 
df_top= df.groupby('DATE').apply(lambda x: x.nlargest(int(len(x)*p),'mvel1')).reset_index(drop=True)  
df_bottom = df.groupby('DATE').apply(lambda x: x.nsmallest(int(len(x)*p),'mvel1')).reset_index(drop=True)  


In [ ]:

def calculate_r2(y_true, y_pred, in_sample=True, benchmark=None):
    if in_sample:
        return 1 - (np.sum((y_true - y_pred) ** 2) / 
                    np.sum((y_true - np.mean(y_true)) ** 2))
    else:
        if benchmark is None:
            raise ValueError("Benchmark must be provided for out-of-sample R-squared calculation.")
        return 1 - (np.sum((y_true - y_pred) ** 2) / 
                    np.sum((y_true - benchmark) ** 2))

In [ ]:
features = df.columns[~df.columns.isin(['DATE', 'DATE2', "mvel2",'permno',"permno2",'risk_premium'])].tolist()
df[features]=df.groupby('DATE')[features].rank(pct=True)

df[features] = 2*df[features] - 1

In [ ]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                #    test_period=12,
                   freq='months')

features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()

X = df[features]
y = df[['risk_premium']]

predictions = []
y_val_list = []
dates = []
dic_r2_all = {}


for train_index, val_index in tscv.split(X, first_split_date= datetime.date(1985,1,31), second_split_date= datetime.date(1987,1,31)):


    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]

    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    huber = HuberRegressor(max_iter=1000)

    # fitting the huber model
    huber.fit(X_train, y_train)

    # predicting on the training set and assiging to the training list
    preds_train = huber.predict(X_train)

    preds_val = huber.predict(X_val) 
    predictions.append(preds_val)
    dates.append(y_val.index)
    y_val_list.append(y_val)
    
    #Calculate OOS model performance the for current window
    r2_train = 1-np.sum(pow(y_train['risk_premium']-preds_train,2))/np.sum(pow(y_train['risk_premium'],2))
    r2_val = 1-np.sum(pow(y_val['risk_premium']-preds_val,2))/np.sum(pow(y_val['risk_premium'],2))
    #Save OOS model performance and the respective month to dictionary
    dic_r2_all["r2." + str(y_val['risk_premium'].index)] = r2_val
    

    print(f'R2 {y_train.index[0][0].date()} - {y_train.index[-1][0].date()} training set {r2_train}')
    print(f'R2 {y_val.index[0][0].date()} - {y_val.index[-1][0].date()} validation set {r2_val}')
    print('---')

predictions_all_full = np.concatenate(predictions, axis=0)
y_test_list_all_full = np.concatenate(y_val_list, axis=0)
dates_all_full = np.concatenate(dates, axis=0)

# R2FULL = 1-np.sum(pow(y_test_list_all_full-predictions_all_full,2))/np.sum(pow(y_test_list_all_full,2))
# print("R2OOS Linear Regression: ", R2FULL)
R2FULL = r2_score(y_test_list_all_full, predictions_all_full)
print("R2OOS Linear Regression: ", R2FULL)

In [ ]:

tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   freq='months')

features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()

X = df_top[features]
y = df_top[['risk_premium']]

predictions_top = []
y_val_list_top = []
dates_top = []
dic_r2_all_top = {}


for train_index, val_index in tscv.split(X, first_split_date= datetime.date(1985,1,31), second_split_date= datetime.date(1987,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]

    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    huber = HuberRegressor(max_iter=1000)

    # fitting the huber model
    huber.fit(X_train, y_train)

    # predicting on the training set and assiging to the training list
    preds_train = huber.predict(X_train)

    preds_val = huber.predict(X_val) 
    predictions_top.append(preds_val)
    dates_top.append(y_val.index)
    y_val_list_top.append(y_val)
    
    #Calculate OOS model performance the for current window
    r2_train = 1-np.sum(pow(y_train['risk_premium']-preds_train,2))/np.sum(pow(y_train['risk_premium'],2))
    r2_val = 1-np.sum(pow(y_val['risk_premium']-preds_val,2))/np.sum(pow(y_val['risk_premium'],2))
    #Save OOS model performance and the respective month to dictionary
    dic_r2_all_top["r2." + str(y_val['risk_premium'].index)] = r2_val
    

    print(f'R2  training set {r2_train}')
    print(f'R2  validation set {r2_val}')
    print('---')

predictions_all_full_top = np.concatenate(predictions_top, axis=0)
y_test_list_all_full_top = np.concatenate(y_val_list_top, axis=0)
dates_all_full_top = np.concatenate(dates_top, axis=0)

# R2TOP = 1-np.sum(pow(y_test_list_all_full_top-predictions_all_full_top,2))/np.sum(pow(y_test_list_all_full_top,2))
# print("R2OOS Linear Regression: ", R2TOP)


R2TOP = r2_score(y_test_list_all_full_top, predictions_all_full_top)
print("R2OOS Linear Regression: ", R2TOP)

In [ ]:
tscv = TimeBasedCV(train_period=60,
                   val_period=24,
                   test_period=12,
                   freq='months')


features = df.columns[~df.columns.isin(['permno', 'permno2', 'mvel12', 'DATE2', 'risk_premium'])].tolist()
X = df_bottom[features]
y = df_bottom[['risk_premium']]

#Empty containers to save results from each window

predictions_bottom = []
y_val_list_bottom =[]
dates_bottom = []
dic_r2_all_bottom = {}


for train_index, val_index in tscv.split(X, first_split_date= datetime.date(1985,1,31), second_split_date= datetime.date(1987,1,31)):

    X_train   = X.loc[train_index].drop('DATE', axis=1)
    y_train = y.loc[train_index]

    X_val   = X.loc[val_index].drop('DATE', axis=1)
    y_val = y.loc[val_index]

    huber = HuberRegressor(max_iter=1000)

    # fitting the huber model
    huber.fit(X_train, y_train)

    # predicting on the training set and assiging to the training list
    preds_train = huber.predict(X_train)

    preds_val = huber.predict(X_val) 
    predictions_bottom.append(preds_val)
    dates_bottom.append(y_val.index)
    y_val_list_bottom.append(y_val)
    
    #Calculate OOS model performance the for current window
    r2_train = 1-np.sum(pow(y_train['risk_premium']-preds_train,2))/np.sum(pow(y_train['risk_premium'],2))
    r2_val = 1-np.sum(pow(y_val['risk_premium']-preds_val,2))/np.sum(pow(y_val['risk_premium'],2))
    #Save OOS model performance and the respective month to dictionary
    dic_r2_all_bottom["r2." + str(y_val['risk_premium'].index)] = r2_val
    

    print(f'R2  training set {r2_train}')
    print(f'R2  validation set {r2_val}')
    print('---')

predictions_all_full_bottom = np.concatenate(predictions_bottom, axis=0)
y_test_list_all_full_bottom = np.concatenate(y_val_list_bottom, axis=0)
dates_all_full_bottom = np.concatenate(dates_bottom, axis=0)


R2BOTTOM = r2_score(y_test_list_all_full_top, predictions_all_full_top)
print("R2OOS Linear Regression: ", R2BOTTOM)

In [ ]:
chart = np.array([[R2FULL],
                  [R2TOP],
                  [R2BOTTOM]])

huber = pd.DataFrame(chart, columns=['Huber Regression'],
                     index=['Full Sample', 'Large Firms', 'Small Firms'])

huber

In [ ]:
huber.to_csv(r'huber_model.csv', index=False)

In [ ]:
yhat = predictions_all_full.tolist()
y_true = y_test_list_all_full.tolist()
i = dates_all_full.tolist()

results = pd.DataFrame(
    {'identifier': i,
     'yhat': yhat,
     'y_true': y_true
    })


results["identifier"]= results["identifier"].astype("str")
results["date"] = results["identifier"].str[12:22]
results["id"] = results["identifier"].str[35:40]
results.drop(["identifier"],axis = 1, inplace=True)
results['date'] = pd.to_datetime(results['date'], format='%Y-%m-%d')
results['MonthYear'] = results['date'].dt.to_period('M')
results = results.sort_values(by = ['date', 'id'], ascending = True)
results = results.set_index(['MonthYear','id'])
results.head()


In [ ]:
# results['yhat'] = results['yhat'].apply(lambda x: x[0])
results['y_true'] = results['y_true'].apply(lambda x: x[0])

In [ ]:
data = df[['mvel12', 'macro_tbl', 'macro_svar']].copy()
data.reset_index(inplace=True)
data['permno2'] = data['permno2'].astype('str')
data['MonthYear'] = data['DATE2'].dt.to_period('M')
data.drop('DATE2', axis=1, inplace=True)
data.rename(columns={'permno2': 'id'}, inplace=True)
data.rename(columns={'mvel12': 'market_cap'}, inplace=True)
data.rename(columns={'macro_tbl': 'risk_free_rate'}, inplace=True)
data = data.set_index(['MonthYear','id'])


In [ ]:
bigdata = pd.merge(results, data,left_index=True, right_index=True)
bigdata.reset_index(inplace=True)
bigdata

In [ ]:
bigdata['returns'] = bigdata['y_true'] + bigdata['risk_free_rate']
bigdata

In [ ]:
bigdata['MonthYear1'] = bigdata['MonthYear'].copy()
bigdata['MonthYear'] = bigdata['MonthYear'].astype('int64')
bigdata['NumMonth'] = bigdata['MonthYear'] - 251
bigdata['NumMonth'].unique()

In [ ]:

for i in bigdata['NumMonth'].unique():
    globals()['df_' + str(i)] = bigdata[bigdata['NumMonth'] == i]

In [ ]:

for i in bigdata["NumMonth"].unique():
    globals()['df_' + str(i)]["rank"]= globals()['df_' + str(i)]['yhat'].rank(method='first')
    
for i in bigdata["NumMonth"].unique():
    globals()['df_' + str(i)]["DecileRank"]=pd.qcut(globals()['df_' + str(i)]['rank'].values, 10, labels = False)

#Drop normal rank, retain only decile ranks 
for i in bigdata["NumMonth"].unique():
     globals()['df_' + str(i)].drop('rank', axis=1, inplace=True)

In [ ]:

for i in bigdata["NumMonth"].unique():
    for j,g in globals()['df_' + str(i)].groupby('DecileRank'):
        globals()['df_' + str(i)+ "_" + str(j)] =  g

In [ ]:

for j in np.arange(0,10,1):
    globals()['rank_' + str(j)] = pd.concat([globals()['df_1_'+ str(j)], globals()['df_2_'+ str(j)]], axis=0)
    
# Generate 10 Dataframes for the 10 Decile portfolios 0-9: rank_9: top portfolio, rank_0: bottom portfolio
for i in np.arange(2,348,1):
    for j in np.arange(0,10,1):
        globals()['rank_' + str(j)] = pd.concat([globals()['rank_' + str(j)], globals()['df_' + str(i+1)+ "_" + str(j)]], axis = 0)

In [ ]:

rank_9

In [ ]:
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]["eq_weights"] = 1/globals()['rank_' + str(j)].groupby('MonthYear')["id"].transform('size')

In [ ]:

for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['excess_return_stock_ew'] = globals()['rank_' + str(j)]["y_true"]*globals()['rank_' + str(j)]["eq_weights"]

In [ ]:
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['return_stock_ew'] = globals()['rank_' + str(j)]["returns"]*globals()['rank_' + str(j)]["eq_weights"]

In [ ]:

for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['excess_return_portfolio_ew'] = globals()['rank_' + str(j)].groupby('MonthYear')["excess_return_stock_ew"].transform('sum')

In [ ]:

for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['return_portfolio_ew'] = globals()['rank_' + str(j)].groupby('MonthYear')["return_stock_ew"].transform('sum')

In [ ]:

for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['pred_excess_return_stock_ew'] = globals()['rank_' + str(j)]["yhat"]*globals()['rank_' + str(j)]["eq_weights"]

In [ ]:
for j in np.arange(0,10,1):
    globals()['rank_' + str(j)]['pred_excess_return_portfolio_ew'] = globals()['rank_' + str(j)].groupby('MonthYear')["pred_excess_return_stock_ew"].transform('sum')

In [ ]:
for j in np.arange(0,10,1):
    globals()['montly_rank_' + str(j)] = globals()['rank_' + str(j)][["MonthYear1", "DecileRank",
                                                                      "excess_return_portfolio_ew",
                                                                      "pred_excess_return_portfolio_ew",
                                                                      "return_portfolio_ew"]]

In [ ]:
for j in np.arange(0,10,1):
    globals()['montly_rank_' + str(j)]=globals()['montly_rank_' + str(j)].drop_duplicates()
    globals()['montly_rank_' + str(j)]=globals()['montly_rank_' + str(j)].set_index("MonthYear1")

In [ ]:
for j in np.arange(0,10,1):
    #Time-series average of realized excess returns
    globals()["ew_mean_return_rank_" +  str(j)]= globals()['montly_rank_' + str(j)]["excess_return_portfolio_ew"].mean()
    #Time-series average of predicted excess returns
    globals()["ew_mean_pred_return_rank_" +  str(j)]= globals()['montly_rank_' + str(j)]["pred_excess_return_portfolio_ew"].mean()
    #Standard deviation of realized excess returns
    globals()["std_ew_rank_" +  str(j)]= globals()['montly_rank_' + str(j)]["excess_return_portfolio_ew"].std()
    #Annualized sharpe ratio of realized excess returns
    globals()["sharpe_ew_rank_" +  str(j)]= (globals()['montly_rank_' + str(j)]["excess_return_portfolio_ew"].mean()/globals()['montly_rank_' + str(j)]["return_portfolio_ew"].std())* np.sqrt(12)

In [ ]:

# For the zero-net-investment long-short portfolio the top (long) and bottom(short) decile portfolios are needed

long_monthly = rank_9[["NumMonth",
                       "MonthYear1",
                       "DecileRank", 
                       "excess_return_portfolio_ew",
                       "pred_excess_return_portfolio_ew",
                       "return_portfolio_ew",
                        ]].drop_duplicates()

short_monthly = rank_0[["NumMonth",
                        "MonthYear1",
                        "DecileRank",
                        "excess_return_portfolio_ew",
                       "pred_excess_return_portfolio_ew",
                       "return_portfolio_ew",
                        ]].drop_duplicates()

# Create a column, indication the stategy 
long_monthly["Strategy"]= "long"
short_monthly["Strategy"]= "short"

# Merge to get the zero net investment portfolio
zeronet_monthly= pd.concat([long_monthly, short_monthly])
zeronet_monthly = zeronet_monthly.sort_values(by = ['NumMonth',"Strategy"])

#Create two new columns containing the exess return of the portfolio and initially set the values to zero.
zeronet_monthly["excess_return_zeronet_ew"] =0


In [ ]:
for i in range(0, len(zeronet_monthly)):
    if zeronet_monthly.iloc[i,7] == "long":
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i, 3]-zeronet_monthly.iloc[i+1, 3]
    else:
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i-1, 3]-zeronet_monthly.iloc[i, 3]
        


zeronet_monthly["pred_excess_return_zeronet_ew"] =0

for i in range(0, len(zeronet_monthly)):
    if zeronet_monthly.iloc[i,7] == "long":
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i, 4]-zeronet_monthly.iloc[i+1, 4]
    else:
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i-1, 4]-zeronet_monthly.iloc[i, 4]


zeronet_monthly["return_zeronet_ew"] =0

for i in range(0, len(zeronet_monthly)):
    if zeronet_monthly.iloc[i,7] == "long":
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i, 5]-zeronet_monthly.iloc[i+1, 5]
    else:
        zeronet_monthly.iloc[i, -1] = zeronet_monthly.iloc[i-1, 5]-zeronet_monthly.iloc[i, 5]

In [ ]:
#Only the measures at portfolio level are needed
zeronet_monthly = zeronet_monthly[['NumMonth', 
                                   'MonthYear1', 
                                   'excess_return_zeronet_ew',
                                   'pred_excess_return_zeronet_ew',
                                   'return_zeronet_ew',
                                   ]].drop_duplicates()

zeronet_monthly

In [ ]:
#Calculate zero-net portfolio performance measures                                            
#Time-series average of realized excess returns                                             
ew_mean_return_zeronet= zeronet_monthly["excess_return_zeronet_ew"].mean()
#Time-series average of predicted excess returns
ew_mean_pred_return_zeronet = zeronet_monthly["pred_excess_return_zeronet_ew"].mean()
#Standard deviation of realized excess returns
std_ew_zeronet = zeronet_monthly["excess_return_zeronet_ew"].std()
#Annualized sharpe ratio of realized excess returns
sharpe_ew_zeronet = (zeronet_monthly["excess_return_zeronet_ew"].mean()/zeronet_monthly["return_zeronet_ew"].std())* np.sqrt(12)

In [ ]:
zeronet_monthly['excess_return_zeronet_ew']

In [ ]:
chart_np = np.array([[ew_mean_pred_return_rank_0, ew_mean_return_rank_0, std_ew_rank_0, sharpe_ew_rank_0],
                     [ew_mean_pred_return_rank_1, ew_mean_return_rank_1, std_ew_rank_1, sharpe_ew_rank_1],
                     [ew_mean_pred_return_rank_2, ew_mean_return_rank_2, std_ew_rank_2, sharpe_ew_rank_2],
                     [ew_mean_pred_return_rank_3, ew_mean_return_rank_3, std_ew_rank_3, sharpe_ew_rank_3],
                     [ew_mean_pred_return_rank_4, ew_mean_return_rank_4, std_ew_rank_4, sharpe_ew_rank_4],
                     [ew_mean_pred_return_rank_5, ew_mean_return_rank_5, std_ew_rank_5, sharpe_ew_rank_5],
                     [ew_mean_pred_return_rank_6, ew_mean_return_rank_6, std_ew_rank_6, sharpe_ew_rank_6],
                     [ew_mean_pred_return_rank_7, ew_mean_return_rank_7, std_ew_rank_7, sharpe_ew_rank_7],
                     [ew_mean_pred_return_rank_8, ew_mean_return_rank_8, std_ew_rank_8, sharpe_ew_rank_8],
                     [ew_mean_pred_return_rank_9, ew_mean_return_rank_9, std_ew_rank_9, sharpe_ew_rank_9],
                     [ew_mean_pred_return_zeronet, ew_mean_return_zeronet, std_ew_zeronet, sharpe_ew_zeronet]])

ew_df = pd.DataFrame(chart_np, columns=['Pred', 'Real', 'Std', 'Sharpe'],
                              index=['Low (L)', '2', '3', '4', '5','6','7','8',"9",'High (H)', "H-L"])

ew_df['Pred'] = pd.Series(["{0:.2f}%".format(val) for val in ew_df['Pred']], index = ew_df.index)
ew_df['Real'] = pd.Series(["{0:.2f}%".format(val) for val in ew_df['Real']], index = ew_df.index)
ew_df['Std'] = pd.Series(["{0:.2f}%".format(val) for val in ew_df['Std']], index = ew_df.index)
ew_df['Sharpe'] = pd.Series([("%.2f" % round(val, 2)) for val in ew_df['Sharpe']], index = ew_df.index)
ew_df